## Part 1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from pandas.errors import PerformanceWarning
from scipy.optimize import minimize

# 忽略 PerformanceWarning
warnings.simplefilter(action="ignore", category=PerformanceWarning)

## Read data from DailyPrices.csv
data = pd.read_csv('DailyPrices.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data.replace([np.inf, -np.inf], np.nan).dropna()

data_2023 = data.loc['2023-01-01':'2023-12-31']
data_2023 = data_2023.reset_index()
## read the rf rate from rf.csv
rf = pd.read_csv('rf.csv')
rf['Date'] = pd.to_datetime(rf['Date'])
rf.set_index('Date', inplace=True)
rf_2023 = rf.loc['2023-01-01':'2023-12-31']
rf = rf.replace([np.inf, -np.inf], np.nan).dropna()

market_returns = data_2023['SPY'].pct_change().dropna()
rf_2023 = rf_2023['rf'].iloc[:len(market_returns)].values
excess_market_returns = market_returns - rf_2023
excess_market_returns = (market_returns - rf_2023).dropna()

X = sm.add_constant(excess_market_returns)

Models = {}
for stock in data_2023.columns[1:]:
    stock_returns = data_2023[stock].pct_change().dropna()  # stock returns
    excess_stock_returns = (stock_returns - rf_2023).dropna() # excess stock returns
    model = sm.OLS(excess_stock_returns, X.loc[excess_stock_returns.index]).fit()
    Models[stock] = model
def carino_k(returns):
    daily_R=returns
    R= (daily_R+1).prod()-1
    daily_GR=np.log(daily_R+1)
    GR = np.sum(daily_GR)
    K = GR / R
    kt=daily_GR/K/daily_R
    kt =np.nan_to_num(kt, nan=0.0, posinf=0.0, neginf=0.0)
    return kt
    
ini_portfolio = pd.read_csv('initial_portfolio.csv')

holding_period = data.loc['2023-12-29':'2025-01-03']
period_rf = rf.loc['2024-01-02':'2025-01-03']
period_rf = period_rf.replace([np.inf, -np.inf], np.nan).dropna()
total_portfolio=pd.DataFrame()
total_portfolio['Date'] = holding_period.index
total_portfolio.set_index('Date', inplace=True)
portfolio_results = {}
initial_total_value={}
for portfolio_name in ['A', 'B', 'C']:
    portfolio_data = ini_portfolio[ini_portfolio['Portfolio'] == portfolio_name]
    results = []
    portfolio=pd.DataFrame()
    portfolio['Date'] = holding_period.index
    portfolio.set_index('Date', inplace=True)
    

    market_returns = holding_period['SPY'].pct_change().dropna()
    excess_market_returns = market_returns - period_rf['rf']
    ## calculate the total return of each stock in the portfolio
    for _, row in portfolio_data.iterrows():
        stock = row['Symbol']
        shares = row['Holding']

        portfolio[stock] = holding_period[stock] * shares
    
    portfolio['Total']= portfolio.sum(axis=1)
    kt=carino_k(portfolio['Total'].pct_change().dropna())
    for _, row in portfolio_data.iterrows():
        portfolio_sys_ret=0
        stock = row['Symbol']
        shares = row['Holding']
        model = Models[stock]
        if stock not in total_portfolio.columns:
            total_portfolio[stock] = holding_period[stock] * shares
        else:
            total_portfolio[stock] += holding_period[stock] * shares
        
        
        ## return of holing period
        returns = holding_period[stock].pct_change().dropna()

        
        ## systematic return
        alpha = model.params['const']
        beta = model.params['SPY']
        systematic_returns = beta * excess_market_returns
        
        ## calculate total return, systematic contribution, and idiosyncratic contribution
        total_return = (returns + 1).prod() - 1
        
        systematic_return = np.dot(kt, systematic_returns)  
        portfolio_sys_ret += systematic_return
        rf_attribution = np.dot(kt, period_rf['rf'])
        idiosyncratic_contribution = total_return - systematic_return - rf_attribution
        
        results.append({
            'Stock': stock,
            'Total Return': total_return,
            'Systematic Contribution': systematic_return,
            'Rf Return': rf_attribution,
            'Idiosyncratic Contribution': idiosyncratic_contribution,
            'Risk': returns.std(),
            'Systematic Risk': systematic_returns.std()

        })

    initial_total_value[portfolio_name] = portfolio['Total'].iloc[0]
    portfolio['Total Beta'] = sum(Models[stock].params['SPY'] * portfolio[stock] / portfolio['Total'] for stock in portfolio.columns[0:-1])
    ## OLS regress the portfolio returns to market returns HERE
    portfolio_returns = portfolio['Total'].pct_change().dropna()- period_rf['rf']
    market_returns = holding_period['SPY'].pct_change().dropna()- period_rf['rf']
    aligned = portfolio_returns.align(market_returns, join='inner')
    y = aligned[0]
    X = sm.add_constant(aligned[1])
    ols_result = sm.OLS(y, X).fit()
    beta = ols_result.params.iloc[0]
    
    sigma_p=portfolio['Total'].pct_change().std()
    portfolio['Systematic Return'] = (portfolio['Total Beta'] * excess_market_returns)
    total_returns = portfolio['Total'].pct_change().dropna()
    total_return = (total_returns + 1).prod() - 1
    systematic_contribution = np.dot(kt,portfolio['Systematic Return'].iloc[1:])
    total_rf_return = np.dot(carino_k(total_returns), period_rf['rf'])
    idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return
    
    
    
    ## create a DataFrame to store the results
    portfolio_summary = pd.DataFrame(results)
    portfolio_summary.loc['Total'] = {
        'Stock': 'Portfolio',
        'Total Return': total_return,
        'Systematic Contribution': systematic_contribution,
        'Rf Return': total_rf_return,
        'Idiosyncratic Contribution': idiosyncratic_contribution,
        'Risk': sigma_p,
        'Systematic Risk': sigma_p * beta

    }
   
    portfolio_results[portfolio_name] = portfolio_summary

total_portfolio['Total']= total_portfolio.sum(axis=1)
total_portfolio['Total Beta'] = sum(Models[stock].params['SPY'] * total_portfolio[stock] / total_portfolio['Total'] for stock in total_portfolio.columns[0:-1])
total_portfolio['Systematic Return'] = total_portfolio['Total Beta'] * excess_market_returns

total_returns = total_portfolio['Total'].pct_change().dropna()
total_return = (total_returns + 1).prod() - 1
systematic_contribution = np.sum(np.dot(carino_k(total_return),total_portfolio['Systematic Return'].iloc[1:]))
total_rf_return = np.sum(np.dot(carino_k(total_returns), period_rf['rf']))
idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return

## regress
portfolio_returns = total_portfolio['Total'].pct_change().dropna()- period_rf['rf']
market_returns = holding_period['SPY'].pct_change().dropna()- period_rf['rf']
aligned = portfolio_returns.align(market_returns, join='inner')
y = aligned[0]
X = sm.add_constant(aligned[1])
ols_result = sm.OLS(y, X).fit()
beta = ols_result.params.iloc[0]


sigma_p=total_portfolio['Total'].pct_change().std()
portfolio_results['Total Portfolio']= pd.DataFrame({
    'Stock': 'Total Portfolio of A,B,C',
    'Total Return': total_return,
    'Systematic Contribution': systematic_contribution,
    'Rf Return': total_rf_return,
    'Idiosyncratic Contribution': idiosyncratic_contribution,
    'Risk': sigma_p,
    'Systematic Risk': beta * sigma_p

}, index=['Total'])
## calculate total portfolio results

## print the results
for portfolio_name, summary in portfolio_results.items():
    print(f"Portfolio {portfolio_name} Results:")
    print(summary.loc['Total'])
    print("\n")

Portfolio A Results:
Stock                         Portfolio
Total Return                   0.136642
Systematic Contribution        0.194457
Rf Return                      0.054346
Idiosyncratic Contribution    -0.112161
Risk                           0.007418
Systematic Risk               -0.000002
Name: Total, dtype: object


Portfolio B Results:
Stock                         Portfolio
Total Return                   0.203526
Systematic Contribution        0.185674
Rf Return                      0.055949
Idiosyncratic Contribution    -0.038097
Risk                           0.006867
Systematic Risk                    -0.0
Name: Total, dtype: object


Portfolio C Results:
Stock                         Portfolio
Total Return                   0.281172
Systematic Contribution        0.203779
Rf Return                      0.057783
Idiosyncratic Contribution     0.019611
Risk                           0.007924
Systematic Risk                0.000001
Name: Total, dtype: object


Portfolio 

In [2]:
from IPython.display import display

for portfolio_name, total_summary in portfolio_results.items():
    if portfolio_name == "Total Portfolio":
        break

    total_summary = total_summary[:-1]
    contribution_table = total_summary[['Systematic Contribution', 'Idiosyncratic Contribution']]
    
    print(f"\nPortfolio {portfolio_name} Performance Summary:")
    display(contribution_table)



Portfolio A Performance Summary:


,Systematic Contribution,Idiosyncratic Contribution
0,0.222093,0.210656
1,0.217424,0.168366
2,0.298368,0.122803
3,0.288094,-0.229653
4,0.062443,-0.032701
5,0.074614,-0.057231
6,0.066833,-0.179678
7,0.236636,0.295721
8,0.109662,-0.057485
9,0.128126,-0.174991



Portfolio B Performance Summary:


,Systematic Contribution,Idiosyncratic Contribution
0,0.242214,0.342865
1,0.181787,-0.139916
2,0.355304,0.303460
3,0.264903,0.488731
4,0.067137,0.407334
5,0.088909,0.203153
6,0.176235,0.232187
7,0.130191,-0.185599
8,0.131365,0.113419
9,0.214732,-0.060271



Portfolio C Performance Summary:


,Systematic Contribution,Idiosyncratic Contribution
0,0.103449,0.238308
1,0.256981,-0.165299
2,0.178387,0.043369
3,0.291217,0.026321
4,0.218843,0.320206
5,0.110535,0.400442
6,0.058568,-0.126680
7,0.077112,0.246902
8,0.172574,-0.011981
9,0.220201,0.254675


## Part 2

In [3]:
from scipy.optimize import minimize

expected_rf = rf.loc[:'2023-12-28']['rf'].mean()
expected_market_return = data.loc[:'2023-12-28']['SPY'].pct_change().mean()
expected_returns = {}

for stock, model in Models.items():
    beta = model.params['SPY']
    expected_returns[stock] = expected_rf + beta * (expected_market_return - expected_rf)

Portfolio_weights = {}

def negative_sharpe(weights, sub_expected_returns, cov_matrix, rf):
    portfolio_return = np.dot(weights, list(sub_expected_returns.values()))
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -(portfolio_return - rf) / portfolio_volatility

for portfolio_name in ['A', 'B', 'C']:
    sub_expected_returns = {}
    sub_portfolio_stocks = ini_portfolio[ini_portfolio['Portfolio'] == portfolio_name]['Symbol'].tolist()
    sub_portfolio_returns = data[sub_portfolio_stocks].loc[:'2023-12-28'].pct_change().dropna()
    cov_matrix = sub_portfolio_returns.cov()
    
    for stock in sub_portfolio_stocks:
        sub_expected_returns[stock] = expected_returns[stock]

    num_stocks = len(sub_portfolio_stocks)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) 
    bounds = [(0, 1) for _ in range(num_stocks)]
    initial_weights = np.array([1/num_stocks] * num_stocks)

    optimal_result = minimize(
        negative_sharpe, 
        initial_weights, 
        args=(sub_expected_returns, cov_matrix, expected_rf),
        method='SLSQP', 
        bounds=bounds, 
        constraints=constraints
    )

    Portfolio_weights[portfolio_name] = optimal_result.x

for portfolio_name, weights in Portfolio_weights.items():
    print(f"Optimal Weights for Portfolio {portfolio_name}:")
    for stock, weight in zip(ini_portfolio[ini_portfolio['Portfolio'] == portfolio_name]['Symbol'].tolist(), weights):
        print(f"{stock}: {weight:.4f}")
    print("\n")

optimal_portfolio_results = {}

for portfolio_name, weights in Portfolio_weights.items():
    sub_portfolio_stocks = ini_portfolio[ini_portfolio['Portfolio'] == portfolio_name]['Symbol'].tolist()
    results = []
    portfolio = pd.DataFrame(index=holding_period.index)

    for i, stock in enumerate(sub_portfolio_stocks):
        weight = weights[i]
        initial_value = initial_total_value[portfolio_name]
        portfolio[stock] = initial_value * weight / holding_period[stock].iloc[0] * holding_period[stock]

    portfolio['Total'] = portfolio.sum(axis=1)

    for stock in sub_portfolio_stocks:
        model = Models[stock]
        beta = model.params['SPY']
        returns = holding_period[stock].pct_change().dropna()
        market_returns = holding_period['SPY'].pct_change().dropna()
        rf_holding = period_rf['rf']
        excess_market_returns = market_returns - rf_holding
        systematic_returns = beta * excess_market_returns

        total_return = (returns + 1).prod() - 1
        systematic_contribution = np.sum(np.dot(carino_k(returns), systematic_returns))
        total_rf_return = np.sum(np.dot(carino_k(returns), rf_holding))
        idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return

        results.append({
            'Stock': stock,
            'Total Return': total_return,
            'Systematic Contribution': systematic_contribution,
            'Rf Return': total_rf_return,
            'Idiosyncratic Contribution': idiosyncratic_contribution,
            'Risk': returns.std(),
            'Systematic Risk': systematic_returns.std()
        })

    portfolio['Total Beta'] = sum(
        Models[stock].params['SPY'] * portfolio[stock] / portfolio['Total'] 
        for stock in sub_portfolio_stocks
    )
    
    portfolio['Systematic Return'] = portfolio['Total Beta'] * excess_market_returns
    total_rf_return = np.sum(np.dot(carino_k(portfolio['Total'].pct_change().dropna()), period_rf['rf']))
    total_return = portfolio['Total'].iloc[-1] / initial_total_value[portfolio_name] - 1
    systematic_contribution = np.sum(np.dot(carino_k(portfolio['Total'].pct_change().dropna()), portfolio['Systematic Return'].iloc[1:]))
    idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return
    
    portfolio_returns = portfolio['Total'].pct_change().dropna() - period_rf['rf']
    aligned = portfolio_returns.align(market_returns, join='inner')
    ols_result = sm.OLS(aligned[0], sm.add_constant(aligned[1])).fit()
    beta = ols_result.params.iloc[0]
    sigma_p = portfolio['Total'].pct_change().std()

    portfolio_summary = pd.DataFrame(results)
    portfolio_summary.loc['Total'] = {
        'Stock': 'Portfolio',
        'Total Return': total_return,
        'Systematic Contribution': systematic_contribution,
        'Rf Return': total_rf_return,
        'Idiosyncratic Contribution': idiosyncratic_contribution,
        'Risk': sigma_p,
        'Systematic Risk': sigma_p * beta
    }

    optimal_portfolio_results[portfolio_name] = portfolio_summary


Optimal Weights for Portfolio A:
WFC: 0.0179
ETN: 0.0377
AMZN: 0.0918
QCOM: 0.0112
LMT: 0.0268
KO: 0.0567
JNJ: 0.0233
ISRG: 0.0427
XOM: 0.0000
MDT: 0.0000
DHR: 0.0194
PLD: 0.0329
BA: 0.0158
PG: 0.0763
MRK: 0.0466
AMD: 0.0102
BX: 0.0422
PM: 0.0424
SCHW: 0.0000
VZ: 0.0125
COP: 0.0330
ADI: 0.0522
BAC: 0.0315
NOW: 0.0478
TMO: 0.0123
CVX: 0.0305
ANET: 0.0104
NVDA: 0.0512
GE: 0.0128
GILD: 0.0261
MU: 0.0008
CMCSA: 0.0636
DIS: 0.0215


Optimal Weights for Portfolio B:
AXP: 0.0215
HON: 0.0500
META: 0.0322
NFLX: 0.0158
PGR: 0.0000
LLY: 0.0207
JPM: 0.0208
VRTX: 0.0171
TJX: 0.0548
EQIX: 0.0363
AAPL: 0.1193
FI: 0.0305
DE: 0.0309
SBUX: 0.0217
GOOGL: 0.0542
T: 0.0200
ABT: 0.0251
BMY: 0.0151
MS: 0.0618
CRM: 0.0235
PFE: 0.0119
SPGI: 0.0170
BRK-B: 0.0438
ADBE: 0.0440
ACN: 0.0280
AMGN: 0.0106
LIN: 0.0170
V: 0.0097
WMT: 0.0479
AMAT: 0.0486
CAT: 0.0114
RTX: 0.0078
UNP: 0.0306


Optimal Weights for Portfolio C:
IBM: 0.0121
TXN: 0.0683
ADP: 0.0314
GOOG: 0.0664
ORCL: 0.0142
BSX: 0.0084
UNH: 0.0357
TMUS: 0.012

In [4]:
# Initialize a dictionary to store results for optimal portfolios
optimal_portfolio_results = {}

# Define a function to print portfolio performance in a formatted way
def print_portfolio_performance(portfolio_results):
    print("\n" + "="*120)
    print("{:<15} {:<20} {:<20} {:<20} {:<20} {:<15} {:<15}".format(
        "Portfolio", "Total Return", "Systematic Contrib", "Idiosyncratic Contrib", 
        "Risk-Free Return", "Total Risk", "Systematic Risk"))
    print("-"*120)
    
    for portfolio_name, summary in portfolio_results.items():
        total = summary.loc['Total']
        print("{:<15} {:<20.2%} {:<20.2%} {:<20.2%} {:<20.2%} {:<15.4f} {:<15.4f}".format(
            portfolio_name,
            total['Total Return'],
            total['Systematic Contribution'],
            total['Idiosyncratic Contribution'],
            total['Rf Return'],
            total['Risk'],
            total['Systematic Risk']
        ))
    
    print("="*120)
    
    # Print detailed stock-level performance for each portfolio
    for portfolio_name, summary in portfolio_results.items():
        print("\n" + "-"*60)
        print(f"DETAILED PERFORMANCE: {portfolio_name} PORTFOLIO".center(60))
        print("-"*60)
        print("{:<15} {:<15} {:<15} {:<15} {:<15}".format(
            "Stock", "Total Return", "Sys Contrib", "Idio Contrib", "Risk"
        ))
        print("-"*60)
        
        for _, row in summary.iterrows():
            if row['Stock'] != 'Portfolio':
                print("{:<15} {:<15.2%} {:<15.2%} {:<15.2%} {:<15.4f}".format(
                    row['Stock'],
                    row['Total Return'],
                    row['Systematic Contribution'],
                    row['Idiosyncratic Contribution'],
                    row['Risk']
                ))
        print("-"*60)
        print("{:<15} {:<15.2%} {:<15.2%} {:<15.2%} {:<15.4f}".format(
            "PORTFOLIO TOTAL",
            summary.loc['Total']['Total Return'],
            summary.loc['Total']['Systematic Contribution'],
            summary.loc['Total']['Idiosyncratic Contribution'],
            summary.loc['Total']['Risk']
        ))
        print("-"*60)

# Process each portfolio
for portfolio_name, weights in Portfolio_weights.items():
    sub_portfolio_stocks = ini_portfolio[ini_portfolio['Portfolio'] == portfolio_name]['Symbol'].tolist()
    results = []
    portfolio = pd.DataFrame()
    portfolio['Date'] = holding_period.index
    portfolio.set_index('Date', inplace=True)

    # Calculate the daily value of the optimal portfolio
    for i, stock in enumerate(sub_portfolio_stocks):
        weight = weights[i]
        initial_value = initial_total_value[portfolio_name]
        portfolio[stock] = initial_value * weight/holding_period[stock].iloc[0] * holding_period[stock]

    # Calculate the total portfolio value
    portfolio['Total'] = portfolio.sum(axis=1)

    # Calculate systematic and idiosyncratic contributions for each stock
    for stock in sub_portfolio_stocks:
        model = Models[stock]
        beta = model.params['SPY']
        returns = holding_period[stock].pct_change().dropna()
        market_returns = holding_period['SPY'].pct_change().dropna()
        rf_holding = period_rf['rf']
        excess_returns = returns - rf_holding
        excess_market_returns = market_returns - rf_holding

        # Systematic returns
        systematic_returns = beta * excess_market_returns

        # Aggregate results
        total_return = (returns + 1).prod() - 1
        systematic_contribution = np.sum(np.dot(carino_k(returns), systematic_returns))
        total_rf_return = np.sum(np.dot(carino_k(returns), rf_holding))
        idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return

        results.append({
            'Stock': stock,
            'Total Return': total_return,
            'Systematic Contribution': systematic_contribution,
            'Rf Return': total_rf_return,
            'Idiosyncratic Contribution': idiosyncratic_contribution,
            'Risk': returns.std(),
            'Systematic Risk': systematic_returns.std()
        })

    # Calculate portfolio-level metrics
    portfolio['Total Beta'] = sum(
        Models[stock].params['SPY'] * portfolio[stock] / portfolio['Total'] for stock in sub_portfolio_stocks
    )
    portfolio['Systematic Return'] = portfolio['Total Beta'] * excess_market_returns
    total_rf_return = np.sum(np.dot(carino_k(portfolio['Total'].pct_change().dropna()), period_rf['rf']))

    # Aggregate portfolio-level results
    total_return = portfolio['Total'].iloc[-1] / initial_total_value[portfolio_name] - 1
    systematic_contribution = np.sum(np.dot(carino_k(portfolio['Total'].pct_change().dropna()), portfolio['Systematic Return'].iloc[1:]))
    idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return
    
    # OLS regression
    portfolio_returns = portfolio['Total'].pct_change().dropna() - period_rf['rf']
    market_returns = holding_period['SPY'].pct_change().dropna() - period_rf['rf']
    aligned = portfolio_returns.align(market_returns, join='inner')
    y = aligned[0]
    X = sm.add_constant(aligned[1])
    ols_result = sm.OLS(y, X).fit()
    beta = ols_result.params[0]
    
    sigma_p = portfolio['Total'].pct_change().std()

    portfolio_summary = pd.DataFrame(results)
    portfolio_summary.loc['Total'] = {
        'Stock': 'Portfolio',
        'Total Return': total_return,
        'Systematic Contribution': systematic_contribution,
        'Rf Return': total_rf_return,
        'Idiosyncratic Contribution': idiosyncratic_contribution,
        'Risk': sigma_p,
        'Systematic Risk': sigma_p * beta
    }

    optimal_portfolio_results[portfolio_name] = portfolio_summary

# Print the optimized output
print("\n" + "="*80)
print("OPTIMAL PORTFOLIO PERFORMANCE ANALYSIS".center(80))
print("="*80)
print_portfolio_performance(optimal_portfolio_results)


                     OPTIMAL PORTFOLIO PERFORMANCE ANALYSIS                     

Portfolio       Total Return         Systematic Contrib   Idiosyncratic Contrib Risk-Free Return     Total Risk      Systematic Risk
------------------------------------------------------------------------------------------------------------------------
A               28.82%               22.28%               0.74%                5.80%                0.0080          0.0078         
B               25.88%               20.78%               -0.63%               5.73%                0.0074          0.0064         
C               30.50%               21.47%               3.19%                5.83%                0.0082          0.0080         

------------------------------------------------------------
             DETAILED PERFORMANCE: A PORTFOLIO              
------------------------------------------------------------
Stock           Total Return    Sys Contrib     Idio Contrib    Risk           
---

## Part 4/5

In [5]:
from scipy.stats import norm, skewnorm, jf_skew_t, norminvgauss
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.optimize import minimize
from scipy.special import gamma
# Define a function for Normal Inverse Gaussian (NIG) PDF
def fit_norminvgauss(data):
    params = norminvgauss.fit(data)
    return params
# Fit Skew Normal distribution
def fit_skewnorm(data):
    params = skewnorm.fit(data)
    return params

# Fit Generalized T distribution
def fit_generalized_t(data):
    params = jf_skew_t.fit(data)
    return params

# Fit Normal distribution
def fit_normal(data):
    params = norm.fit(data)
    return params

# Evaluate AICc
def calculate_aicc(log_likelihood, num_params, num_data):
    aic = 2 * num_params - 2 * log_likelihood
    aicc = aic + (2 * num_params * (num_params + 1)) / (num_data - num_params - 1)
    return aicc

# Fit all distributions and choose the best fit
def fit_distributions(data):
    results = {}
    num_data = len(data)
    
    # Fit Normal
    normal_params = fit_normal(data)
    normal_loglik = np.sum(norm.logpdf(data, *normal_params))
    results['Normal'] = {
        'params': normal_params,
        'log_likelihood': normal_loglik,
        'aicc': calculate_aicc(normal_loglik, len(normal_params), num_data)
    }
    
    # Fit Skew Normal
    skewnorm_params = fit_skewnorm(data)
    skewnorm_loglik = np.sum(skewnorm.logpdf(data, *skewnorm_params))
    results['Skew Normal'] = {
        'params': skewnorm_params,
        'log_likelihood': skewnorm_loglik,
        'aicc': calculate_aicc(skewnorm_loglik, len(skewnorm_params), num_data)
    }
    
    # Fit Generalized T
    generalized_t_params = fit_generalized_t(data)
    generalized_t_loglik = np.sum(jf_skew_t.logpdf(data, *generalized_t_params))
    results['Generalized T'] = {
        'params': generalized_t_params,
        'log_likelihood': generalized_t_loglik,
        'aicc': calculate_aicc(generalized_t_loglik, len(generalized_t_params), num_data)
    }
    
    # Fit NIG
    nig_params = fit_norminvgauss(data)
    if nig_params is not None:
        nig_loglik = np.sum(norminvgauss.logpdf(data, *nig_params))
        results['NIG'] = {
            'params': nig_params,
            'log_likelihood': nig_loglik,
            'aicc': calculate_aicc(nig_loglik, len(nig_params), num_data)
        }
    
    # Choose the best fit based on AICc
    best_fit = min(results.items(), key=lambda x: x[1]['aicc'])
    return best_fit, results

# Example: Fit distributions for each stock
  # Pre-holding period data

stock_results = {}
data1= data.loc[:'2023-12-29']
for stock in data.columns:
    stock_data = data1[stock].dropna().pct_change().dropna()
    best_fit, all_results = fit_distributions(stock_data)
    stock_results[stock] = {
        'best_fit': best_fit,
        'all_results': all_results
    }

# Print results
print("="*80)
print("{:<10} {:<15} {:<40} {:<15}".format("Stock", "Best Fit", "Parameters", "AICc"))
print("-"*80)
for stock, result in stock_results.items():
    params_str = ", ".join([f"{p:.4f}" for p in result['best_fit'][1]['params']])
    print("{:<10} {:<15} {:<40} {:<15.2f}".format(
        stock, 
        result['best_fit'][0], 
        params_str, 
        result['best_fit'][1]['aicc']
    ))
print("="*80)
    
import scipy.stats as stats
import mpmath as mp
def CDF(stock):
    if stock_results[stock]['best_fit'][0] == 'Generalized T':
        params = stock_results[stock]['best_fit'][1]['params']
        return stats.jf_skew_t.cdf(data[stock].pct_change().dropna(), *params)
    elif stock_results[stock]['best_fit'][0] == 'Normal':
        params = stock_results[stock]['best_fit'][1]['params']
        return stats.norm.cdf(data[stock].pct_change().dropna(), *params)
    elif stock_results[stock]['best_fit'][0] == 'Skew Normal':
        params = stock_results[stock]['best_fit'][1]['params']
        return stats.skewnorm.cdf(data[stock].pct_change().dropna(), *params)
    elif stock_results[stock]['best_fit'][0] == 'NIG':
        params = stock_results[stock]['best_fit'][1]['params']
        return norminvgauss.cdf(data[stock].pct_change().dropna(), *params)

daily_returns=pd.DataFrame()
daily_returns['Date'] = data.pct_change().dropna().index
daily_returns.set_index('Date', inplace=True)
for stock in data.columns:
    daily_returns[stock] = CDF(stock)
    daily_returns[stock] = norm.ppf(daily_returns[stock])
    
spearman_rank = daily_returns.corr(method='spearman')
np.random.seed(33)
n_samples =  1000
random_samples = np.random.multivariate_normal(np.zeros(len(daily_returns.columns)), spearman_rank, size=n_samples)
random_samples = pd.DataFrame(random_samples, columns=daily_returns.columns)
random_samples = norm.cdf(random_samples)
## the processor always crush here, so I do it separately. Yet found no problem.
random_samples = pd.DataFrame(random_samples, columns=daily_returns.columns)
def ppf(data, stock):
    data[stock]=np.clip(data[stock], 1e-8, 1-1e-8)
    if stock_results[stock]['best_fit'][0] == 'Generalized T':
        params = stock_results[stock]['best_fit'][1]['params']
        return stats.jf_skew_t.ppf(data[stock], *params)
    elif stock_results[stock]['best_fit'][0] == 'Normal':
        params = stock_results[stock]['best_fit'][1]['params']
        return stats.norm.ppf(data[stock], *params)
    elif stock_results[stock]['best_fit'][0] == 'Skew Normal':
        params = stock_results[stock]['best_fit'][1]['params']
        return stats.skewnorm.ppf(data[stock], *params)
    elif stock_results[stock]['best_fit'][0] == 'NIG':
        params = stock_results[stock]['best_fit'][1]['params']
        try: 
            values=norminvgauss.ppf(data[stock], *params)
            return values
        except:
            values = np.zeros(len(data[stock]))
            return values
temp=ppf(random_samples, 'PLTR')
for stock in random_samples.columns:
    print(stock)
    random_samples[stock] = ppf(random_samples, stock)

portfolio_A = ini_portfolio[ini_portfolio['Portfolio'] == 'A']
symbols_A = portfolio_A['Symbol'].tolist()
holdings_A = dict(zip(portfolio_A['Symbol'], portfolio_A['Holding']))
portfolio_B = ini_portfolio[ini_portfolio['Portfolio'] == 'B']
symbols_B = portfolio_B['Symbol'].tolist()
holdings_B = dict(zip(portfolio_B['Symbol'], portfolio_B['Holding']))
portfolio_C = ini_portfolio[ini_portfolio['Portfolio'] == 'C']
symbols_C = portfolio_C['Symbol'].tolist()
holdings_C = dict(zip(portfolio_C['Symbol'], portfolio_C['Holding']))

A=pd.DataFrame()
A['Date'] = data.index
A.set_index('Date', inplace=True)
A=A.loc[['2023-12-29']]
for stock in symbols_A:
    A[stock] = data[stock] * holdings_A[stock]

B=pd.DataFrame()
B['Date'] = data.index
B.set_index('Date', inplace=True)
B=B.loc[['2023-12-29']]
for stock in symbols_B:
    B[stock] = data[stock] * holdings_B[stock]
C=pd.DataFrame()
C['Date'] = data.index
C.set_index('Date', inplace=True)
C=C.loc[['2023-12-29']]
for stock in symbols_C:
    C[stock] = data[stock] * holdings_C[stock]

### Using the data on 2023-12-29 in Part1
portfolios={'A':A, 'B':B, 'C':C}
def compute_var_es(portfolio_name):
    sample_results =np.zeros_like(random_samples)
    sample_results = pd.DataFrame(sample_results, columns=random_samples.columns)
    if portfolio_name == 'Total':
        current_portfolio = total_portfolio.loc[['2023-12-29']]
        current_portfolio = current_portfolio.drop(columns=['Total', 'Total Beta', 'Systematic Return'])
        for stock in current_portfolio.columns:
            sample_results[stock] = (1+random_samples[stock].values)* current_portfolio[stock].values
    else:
        current_portfolio = portfolios[portfolio_name]
   
        for stock in current_portfolio.columns:

            sample_results[stock] = (1+random_samples[stock].values)* current_portfolio[stock].values
  
    current_value = current_portfolio.sum(axis=1).values[0]
    print(f'{portfolio_name} current value: {current_value}')
    sample_results['Total']= sample_results.sum(axis=1)
    var_95 = np.percentile(sample_results['Total'], 5)
    var= current_value - var_95
    es_95 = sample_results[sample_results['Total'] <= var_95]['Total'].mean()
    es = current_value - es_95
    return var, es

print("\n" + "="*60)
print("{:<10} {:<15} {:<15} {:<15}".format("Portfolio", "Current Value", "VaR (5%)", "ES (5%)"))
print("-"*60)
for portfolio_name in ['A', 'B', 'C', 'Total']:
    var, es = compute_var_es(portfolio_name)
    current_value = portfolios[portfolio_name].sum(axis=1).values[0] if portfolio_name != 'Total' else total_portfolio.loc[['2023-12-29']].drop(columns=['Total', 'Total Beta', 'Systematic Return']).sum(axis=1).values[0]
    print("{:<10} {:<15.2f} {:<15.2f} {:<15.2f}".format(
        portfolio_name, 
        current_value,
        var,
        es
    ))
print("="*60)

from scipy.linalg import eigh
from scipy.linalg import sqrtm
from numpy.linalg import norm
daily_returns = data.pct_change().dropna()
cov_matrix = daily_returns.cov()
std_devs=np.sqrt(np.diag(cov_matrix))
init_corr_matrix = np.diag(1/std_devs) @ cov_matrix @ np.diag(1/std_devs) 
def projection1(matrix):
    matrix1 = matrix.copy()
    np.fill_diagonal(matrix1, 1)
    return matrix1
    

def projection2(matrix):
    eigvals, eigvecs = eigh(matrix)
    eigvals[eigvals < 0] = 1e-8
    eigvals = np.diag(eigvals)
    return eigvecs @ eigvals @ eigvecs.T #weight as 1, neglected

std_devs=np.sqrt(np.diag(cov_matrix))

init_corr_matrix = np.diag(1/std_devs) @ cov_matrix @ np.diag(1/std_devs)

def from_corr_to_cov(corr_matrix, std_devs):
    return np.diag(std_devs) @ corr_matrix @ np.diag(std_devs)

tol=1e-12
max_iter=1000
def higham_psd(matrix):
    global tol, max_iter
    Yk=matrix
    delta_Sk=np.zeros(matrix.shape)
    for i in range(max_iter):
        Rk=Yk-delta_Sk
        Xk=projection2(Rk)
        delta_Sk=Xk-Rk
        if norm(Yk-projection1(Xk), 'fro')<tol:
            Yk=projection1(Xk)
            print("iteration",i)
            break
        Yk=projection1(Xk)
    return Yk   

Stock      Best Fit        Parameters                               AICc           
--------------------------------------------------------------------------------
SPY        Normal          0.0010, 0.0082                           -1679.69       
AAPL       NIG             2.4805, 0.1243, 0.0008, 0.0197           -1474.56       
NVDA       Generalized T   4.0256, 2.6217, -0.0105, 0.0216          -1089.25       
MSFT       Generalized T   4.7166, 3.9338, -0.0025, 0.0136          -1359.75       
AMZN       Generalized T   3.1738, 2.8107, -0.0008, 0.0168          -1230.60       
META       Generalized T   1.7719, 1.2620, -0.0034, 0.0143          -1230.47       
GOOGL      NIG             1.0998, 0.0608, 0.0009, 0.0198           -1285.53       
AVGO       NIG             1.6296, 0.6646, -0.0067, 0.0221          -1272.83       
TSLA       NIG             1.8569, 0.2554, -0.0023, 0.0447          -991.83        
GOOG       NIG             1.1778, 0.0490, 0.0011, 0.0206           -1279.88   

NameError: name 'higham_cov_matrix' is not defined

In [6]:
higham_corr_matrix = higham_psd(init_corr_matrix)
higham_cov_matrix = from_corr_to_cov(higham_corr_matrix, std_devs)

L = np.linalg.cholesky(higham_cov_matrix)
n_samples = 1000
z = np.random.randn(n_samples, cov_matrix.shape[0])
samples=[]
for i in z:
    samples.append(L @ i)
samples=pd.DataFrame(samples)
samples.columns = data.columns  


def compute_var_es_multinorm(portfolio_name):
    temp_samples =np.zeros_like(samples)
    temp_samples = pd.DataFrame(temp_samples)
    if portfolio_name == 'Total':
        current_portfolio = total_portfolio.loc[['2023-12-29']]
        current_portfolio = current_portfolio.drop(columns=['Total', 'Total Beta', 'Systematic Return'])
        for stock in current_portfolio.columns:
            temp_samples[stock] = (1+samples[stock].values)* current_portfolio[stock].values
    else:
        current_portfolio = portfolios[portfolio_name]
   
        for stock in current_portfolio.columns:
            temp_samples[stock] = (1+samples[stock].values)* current_portfolio[stock].values
    temp_samples['Total']= temp_samples.sum(axis=1)
    current_value = current_portfolio.sum(axis=1).values[0]
    var_95 = np.percentile(temp_samples['Total'], 5)
    var= current_value - var_95
    es_95 = temp_samples[temp_samples['Total'] <= var_95]['Total'].mean()
    es = current_value - es_95
    return var, es

for portfolio_name in ['A', 'B', 'C', 'Total']:
    var, es = compute_var_es_multinorm(portfolio_name)
    print(f"1 Day 5% for Portfolio {portfolio_name} VaR: {var:.2f}, ES: {es:.2f}")
    
var_95= np.percentile(total_portfolio['Total'], 5)
es_95 = total_portfolio[total_portfolio['Total'] <= var_95]['Total'].mean()
var = total_portfolio['Total'].iloc[0] - var_95
es = total_portfolio['Total'].iloc[0] - es_95
print("Historical Simulation: ")
print(f"1 Day 5% for Portfolio Total VaR: {var:.2f}, ES: {es:.2f}")

def compute_es(weights,portfolio):
    simulated_returns = np.sum(portfolio @ np.diag(weights), axis=0)
    
    var_95 = np.percentile(simulated_returns, 5)
    es_95 = simulated_returns[simulated_returns <= var_95].mean()
    return -es_95

epsilon = 1e-6

def CES(weights, portfolio):
    original_es = compute_es(weights, portfolio)
    gradient = np.zeros_like(weights)
    for i in range(len(weights)):
        perturbed_weights = weights.copy()
        perturbed_weights[i] += epsilon  # Add a small perturbation to the i-th weight
        perturbed_es = compute_es(perturbed_weights, portfolio)
        gradient[i] = (perturbed_es - original_es) / epsilon  # Numerical deriva
    ces = np.diag(weights) @ gradient
    return ces


def objective_function(weights,portfolio):
    ces = CES(weights, portfolio)
    return ces.std()**2

def weight_constraint(weights):
    return np.sum(weights) - 1

def optimize_weights(portfolio):
    num_assets = len(portfolio.columns)
    initial_weights = np.ones(num_assets) / num_assets  
    bounds = [(0, 1) for _ in range(num_assets)]  
    constraints = {'type': 'eq', 'fun': weight_constraint}  
    result = minimize(objective_function, initial_weights,args=(portfolio), bounds=bounds, constraints=constraints)
    return result.x  

optimal_weights = {}

print("\n" + "="*80)
print("{:<15} {:<50} {:<15}".format("Portfolio", "Optimal Weights", "Min ES"))
print("-"*80)
for portfolio_name in ['A', 'B', 'C']:
    temp = random_samples[[stock for stock in portfolios[portfolio_name].columns]]
    optimal_weights[portfolio_name] = optimize_weights(temp)
    weights_str = ", ".join([f"{w:.4f}" for w in optimal_weights[portfolio_name]])
    print("{:<15} {:<50} {:<15.4f}".format(
        portfolio_name,
        weights_str,
        compute_es(optimal_weights[portfolio_name], temp)
    ))
print("="*80)

optimal_portfolio_results = {}
for portfolio_name, weights in optimal_weights.items():
    sub_portfolio_stocks = ini_portfolio[ini_portfolio['Portfolio'] == portfolio_name]['Symbol'].tolist()
    results = []
    portfolio = pd.DataFrame()
    portfolio['Date'] = holding_period.index
    portfolio.set_index('Date', inplace=True)

    # Calculate the daily value of the optimal portfolio
    
    for i, stock in enumerate(sub_portfolio_stocks):
        weight = weights[i]
        initial_value = initial_total_value[portfolio_name]
        portfolio[stock] = initial_value * weight/holding_period[stock].iloc[0] * holding_period[stock]

    # Calculate the total portfolio value
    portfolio['Total'] = portfolio.sum(axis=1)

    # Calculate systematic and idiosyncratic contributions for each stock
    for stock in sub_portfolio_stocks:
        model = Models[stock]
        beta = model.params['SPY']
        returns = holding_period[stock].pct_change().dropna()
        market_returns = holding_period['SPY'].pct_change().dropna()
        rf_holding = period_rf['rf']
        excess_returns = returns - rf_holding
        excess_market_returns = market_returns - rf_holding

        # Systematic returns
        systematic_returns = beta * excess_market_returns



        # Aggregate results
        total_return = (returns + 1).prod() - 1
        systematic_contribution = np.sum(np.dot(carino_k(returns), systematic_returns))
        total_rf_return = np.sum(np.dot(carino_k(returns), rf_holding))
        idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return

        results.append({
            'Stock': stock,
            'Total Return': total_return,
            'Systematic Contribution': systematic_contribution,
            'Rf Return': total_rf_return,
            'Idiosyncratic Contribution': idiosyncratic_contribution,
            'Risk': returns.std(),
            'Systematic Risk': systematic_returns.std()
        })

    # Calculate portfolio-level metrics
    portfolio['Total Beta'] = sum(
        Models[stock].params['SPY'] * portfolio[stock] / portfolio['Total'] for stock in sub_portfolio_stocks
    )
    portfolio['Systematic Return'] = portfolio['Total Beta'] * excess_market_returns
    total_rf_return = np.sum(np.dot(carino_k(portfolio['Total'].pct_change().dropna()), period_rf['rf']))

    # Aggregate portfolio-level results
    total_return = portfolio['Total'].iloc[-1] / initial_total_value[portfolio_name] - 1
    systematic_contribution = np.sum(np.dot(carino_k(portfolio['Total'].pct_change().dropna()), portfolio['Systematic Return'].iloc[1:]))
    idiosyncratic_contribution = total_return - systematic_contribution - total_rf_return
    
    ## OLS regress the portfolio returns to market returns HERE
    portfolio_returns = portfolio['Total'].pct_change().dropna()- period_rf['rf']
    market_returns = holding_period['SPY'].pct_change().dropna()- period_rf['rf']
    aligned = portfolio_returns.align(market_returns, join='inner')
    y = aligned[0]
    X = sm.add_constant(aligned[1])
    ols_result = sm.OLS(y, X).fit()
    beta = ols_result.params[0]


    sigma_p=portfolio['Total'].pct_change().std()

    portfolio_summary = pd.DataFrame(results)
    portfolio_summary.loc['Total'] = {
        'Stock': 'Portfolio',
        'Total Return': total_return,
        'Systematic Contribution': systematic_contribution,
        'Rf Return': total_rf_return,
        'Idiosyncratic Contribution': idiosyncratic_contribution,
        'Risk': sigma_p,
        'Systematic Risk': sigma_p * beta
    }

    optimal_portfolio_results[portfolio_name] = portfolio_summary

# Output the results for optimal portfolios
print("\n" + "="*120)
print("PORTFOLIO PERFORMANCE SUMMARY")
print("="*120)
print("{:<10} {:<15} {:<20} {:<20} {:<20} {:<15} {:<15}".format(
    "Portfolio", "Total Return", "Systematic Contrib", "Idiosyncratic Contrib", 
    "Risk-Free Return", "Total Risk", "Systematic Risk"))
print("-"*120)

for portfolio_name, summary in optimal_portfolio_results.items():
    total = summary.loc['Total']
    print("{:<10} {:<15.2%} {:<20.2%} {:<20.2%} {:<20.2%} {:<15.4f} {:<15.4f}".format(
        portfolio_name,
        total['Total Return'],
        total['Systematic Contribution'],
        total['Idiosyncratic Contribution'],
        total['Rf Return'],
        total['Risk'],
        total['Systematic Risk']
    ))
print("="*120)

iteration 0
1 Day 5% for Portfolio A VaR: 3626.53, ES: 4679.65
1 Day 5% for Portfolio B VaR: 3282.68, ES: 4240.80
1 Day 5% for Portfolio C VaR: 3376.62, ES: 4335.76
1 Day 5% for Portfolio Total VaR: 9783.77, ES: 12716.73
Historical Simulation: 
1 Day 5% for Portfolio Total VaR: -10148.13, ES: 2142.27

Portfolio       Optimal Weights                                    Min ES         
--------------------------------------------------------------------------------
A               0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303, 0.0303 0.0100         
B               0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0170, 0.0317, 0.0317, 0.0000, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317, 0.0317